In [ ]:
# !python3 -m spacy download xx_ent_wiki_sm

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
real = pd.read_excel('data/data.xlsx')
# Меняем названия на более приемлемые:

target_shops = [
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"', 'ООО "АГРОТОРГ"', 'ООО "ЛЕНТА"',
   'АО "ДИКСИ ЮГ"', 'ООО "АШАН"', 'АО "ТАНДЕР"', 'ООО "БИЛЛА"', 'ООО "О`КЕЙ"',
    'ООО "АГРОАСПЕКТ"', 'ООО "АТАК"'
]

shops_rename = {
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"': 'ПЕРЕКРЕСТОК',
    'ООО "АГРОТОРГ"': 'ПЯТЕРОЧКА',
    'ООО "ЛЕНТА"': 'ЛЕНТА',
    'АО "ДИКСИ ЮГ"': 'ДИКСИ',
    'ООО "АШАН"': 'АШАН',
    'АО "ТАНДЕР"': 'МАГНИТ',
    'ООО "БИЛЛА"': 'БИЛЛА',
    'ООО "О`КЕЙ"': 'О`КЕЙ',
    'ООО "АГРОАСПЕКТ"': 'ПЯТЕРОЧКА',
    'ООО "АТАК"': 'АТАК'
}

real = real[real['shop_name'].isin(target_shops)].copy()
real['shop_name'] = real['shop_name'].apply(lambda x: shops_rename[x])
real = real.drop_duplicates(subset=['name'])
real.sample(2)

,shop_name,name,quantity,price,sum
60640,ЛЕНТА,Стейк ИНДИЛАЙТ из филе грудки ГВУ охл525,2.0,259.99,519.98
4786,ДИКСИ,ХЛЕБ УКРАИНСКИЙ БЕЗ УП. 700Г Х,1.0,26.99,26.99


In [4]:
real.head()

,shop_name,name,quantity,price,sum
19,ДИКСИ,МИНТАЙ ФИЛЕ Б/К СВ/МОР П/ПАК 8,1.0,199.90,199.90
20,ДИКСИ,БЗМЖ МОЛОКО СТРАНА ВАСИЛЬКИ У/,1.0,49.99,49.99
21,ДИКСИ,СУШКИ ТАРАЛЛИНИ С ЧЕСНОКОМ 180,1.0,29.99,29.99
22,ДИКСИ,ЯЙЦО КУРИНОЕ СТОЛОВОЕ 1КАТЕГОР,1.0,63.99,63.99
23,ДИКСИ,ПЕЧЕНЬЕ ЮБИЛЕЙНОЕ ВИТАМИНИЗИРО,2.0,29.99,59.98


In [5]:
clean = pd.read_csv('clean_data/all_clean.csv')

In [6]:
tmp = pd.read_csv('benchmarks/standard.csv')

In [7]:
def find_weight_or_volume(name: str)-> str:
    """Находит вес или объём товара."""

    pattern1 = r'\d+((\.|\,|x|X|х|Х)?)\d*([а-я]|[А-Я])*$' # ищет вес в конце
    pattern2 = r'\d+\w+' #  ищет вес в середине
    pattern3 = r'(\d+ кг)|(\d+ г)'  # находит `1 кг` или `1 г`
    result = re.search(pattern1, name)
    if result:
        return result.group(0)
    result = re.search(pattern2, name)
    if result:
        return result.group(0)
    result = re.search(pattern3, name)
    return result.group(0) if result else ''

In [8]:
# tmp['Название'].apply(find_weight_or_volume)

In [9]:
tmp['Название'] = tmp['Название'].str.lower()

In [146]:
'huggies/sdfsdf'.replace('/', ' ')

'huggies sdfsdf'

In [147]:
from product_dict import PRODUCT_DICT

class NormalizeNames:
    def __init__(self, name: pd.Series):
        self.df = pd.DataFrame(name.copy()).rename(columns={'name': 'name_norm'})
        self.df['product_norm'] = None
        self.df['brand_norm'] = None
        
        self.slash_brands = {
            'deluxe/': 'deluxe',
            'nemoloko/': 'nemoloko',
            'huggies/': 'huggies',
            'figaro/': 'figaro',
            'кп/': 'красная птица',
            'кд/': 'каждый день',
            'heinz/' : 'heinz',
            'lay`s/': 'lays',
            'моя цена/': 'моя цена',
            'магнит/': 'магнит',
            'rich/': 'rich',
            '/овощная семейка': 'овощная семейка',
            'dr.oetker/': 'dr. oetker',
            'кр/цена': 'красня цена',
            'д/ваня': 'дядя ваня',
            '/орео': 'oreo',
            'дом.в/дер.': 'домик в деревне',
            'geese/': 'geese'
        }

        self.slash_product = {
            'б/йог': 'биойогурт',
            'сух/завтрак': 'сухой завтрак',
            '/пучок': 'пучок',
            'з/щ': 'зубная щётка',
            'йогурт/смусси': 'йогурт',
            'т/бумага': 'туалетная бумага',
            'укроп/петрушк': 'зелень',
            'б/полотенца': 'бумажные полотенца',
            '/сыр': 'сыр',
            'з/паста': 'зубная паста',
            'т/мыло': 'мыло',
            'ж/мыло': 'жидкое мыло',
            'СУХАРИ/': 'сухари/',
            'бум/салф': 'бумажные полотенца',
            'з/п': 'зубная паста',
            'шамп/бал': 'шампунь',
            'с/порошок': 'порошок',
            'ш/р': 'шариковая ручка',
            '/творог': 'творог',
            'т/бум': 'тулетная бумага',
            
        }
        self.dot_brands = {
            'д.в д': 'домик в деревне',
            'д.в.д': 'домик в деревне',
            'хл.сп': 'хлебный спас',
            'к.ц.': 'красная цена',
            'кр.ц.': 'красная цена',
            'кр.цена': 'красная цена',
            'пр!ст': 'просто',
            'п.св': 'первая свежесть',
            'раст': 'растишка',
            'гранд дуэт': 'grand duet',
            'прост.': 'простоквашино',
            'прост ': 'простоквашино',
            'mil gen': 'miller genuine',
            'шарл': 'шарлиз',
            'домашняя кухн': 'домашняя кухня',
            'крикет': 'cricket',
            'домик в дерев': 'домик в деревне',
            'акт.': 'актвия',
            'озор.пчел': 'озорная пчелка',
            'вор.сух': 'воронцовские сухарики',
            'dr.oetker': 'dr.oetker',
            'б.ю.александров': 'б. ю. александров',
            'mr.ricco': 'mr. ricco',
            'dr.korner': 'dr. korner',
            'alp.gold': 'alpen gold',
            'mil.gen.dr' : 'miller',
            'с.прид': 'сады придонья',
            'g.gold': 'gourmet gold',
            'colg.з/п': 'colgate',
            'с.пудовъ': 'с. пудовъ',
            'сибирск.коллекция': 'сибирская коллекция',
            '(данон)': 'danone',
            'lay`': 'lays',
            'кока-кола': 'coca-cola',
            'марс': 'mars',
            'нестле': 'nestle',
            'брест-литовск': 'брест-литовский',
            "lay's": 'lays',
            'хенкель': 'henkel',
            'gl.vil': 'global village',
            'bar': 'barilla',
            'tol': 'tolli',
            'юбил.': 'юбилейное',
            'астор': 'astoria',
            'бар.': 'барни',
            'рот фронт': 'ротфронт',
            'роллт': 'роллтон',
            'верхов.': 'верховье',
            'мист.': 'мистраль',
            'иммун': 'иммунеле',
            'рест.': 'рестория',
            'зд.мен.': 'здоровое меню',
            ' вв ': 'вкусвилл',
            'зелен. линия': 'зеленая линия',
            'комм.': 'коммунарка',
            'волоконовско': 'волоконовское',
            'олвейс': 'always',
            'палмолив': 'palmolive',
            'жатецкий гусь': 'zatecky gus',
            'советские традиц': 'советские традиции',
            'президент': 'president',
            'стародв': 'стародворские',
            'альметте': 'almette',
            'дом в дер': 'домик в деревне',
            'дом дер': 'домик в деревне',
            'дом.в дер.': 'домик в деревне',
            'виола виттр': 'виола виттрока',
            'золот ларец': 'золотой ларец',
            'клинск': 'клинский',
            'добр ': 'добрый',
            'добр. ': 'добрый',
            'рублев': 'рублевский',
            'кремлев': 'кремлевский',
            'гессер': 'gosser',
            'воронц': 'воронцовские',
            'гранола': 'granola',
            'r.sp.': 'ritter sport',
            'sиб.кол.': 'сибирская коллекция',
            'яр.бройл.': 'ярославский бройлер',
            'кузм.': 'кузмино',
            'виола': 'viola',
            'чупа чупс': 'chupa chups',
            'наше солнышк': 'наше солнышко',
            'ролл': 'роллтон',
            'зева': 'zewa',
            'б.ю.алекс.': 'б.ю.александров',
            'мамба': 'mamba',
            'эком.': 'ecomilk',
            'савушк.': 'савушкин хуторок',
            'милка': 'milka',
            'виолетта': 'Violette',
            'марк.пер.': 'маркет перекресток',
            'национ.': 'националь',
            'пр.кав.': 'предгорье кавказа',
            'велк.': 'велком',
            'байс.': 'baisad',
            'ростагроэкс': 'ростагроэкспорт',
            'слав.': 'славянка',
            'варен.': 'варенька',
            'хейнек.': 'heineken',
            'гринфилдголден': 'greenfield',
            'стр.вас.': 'страна васильки',
            'бархатные ручк': 'бархатные ручки',
            'велле': 'velle',
            'терре аллегре': 'terre allegre',
            'зелен.линия': 'зеленая линия',
            'ильина': 'от ильиной',
            'данон': 'danone',
            'юби.': 'юбилейное',
            'больш.кружка': 'большая кружка',
            'рязан.': 'рязаночка',
            'вдох.': 'вдохновение',
            'озби': 'семейка озби',
            'ландлибе': 'landliebe',
            'асен.фер.': 'асеньевская ферма',
            'приос.': 'приосколье',
            'б.ю. алексан': 'б.ю. александров',
            'б.ю.алексан': 'б.ю. александров',
            'б.ю.александров': 'б.ю. александров',
            'б.ю. александров': 'б.ю. александров',
            'азов.кф': 'азовская кф',
            'феликс': 'felix',
            'з.боч': 'золотая бочка',
            'пр.ява': 'принцесса ява',
            'золот.ярлык': 'золотой ярлык',
            'село зел.': 'село зеленое',
            'оral-в': 'оral-b',
            'рог.': 'рогачевъ',
            'золотая симф.': 'золотая симфония',
            'золот стан': 'золотой стандарт',
            'mентос': 'mentos',
            'men.': 'mentos',
            'дан.': 'danone',
            'петр ': 'петр I',
            'ява ': 'ява',
            'лд ': 'LD',
            'люб.': 'любимый',
            'act.': 'activia',
            'сенатор': 'senator',
            'мальборо': 'marlboro',
            'филип моррис': 'philip morris',
            'филлип моррис': 'philip morris',
            'винстон': 'winston',
            'бонд': 'bond',
            'ротманс': 'rothmans',
            'эссе': 'esse',
            'bon': 'bonduelle',
            'магги': 'maggi',
            'чист.лин.': 'чистая линия',
            'рост.': 'ростагроэкспорт',
            'залесск.фермер': 'залесский фермер',
            'kаmis': 'kamis',
            'мистер пропер': 'mr. proper',
            'лейз': 'lays',
            'natur.': 'natural',
            'alw.': 'always',
            'tur.dies.': 'turbo diesel',
            'мдб': 'мясной дом бородина',
            'три короч.': 'три корочки',
            'pur.': 'purina',
            'nest.': 'nestle',
            'bio баланс': 'bio balance',
            'посид.': 'посиделкино',
            'сл.слоб.': 'сладкая слобода',
            'coca-c.': 'coca-cola',
            'хугарден': 'hoegaarden',
            'черног.': 'черноголовка',
            'мерси': 'mersi',
            'чоко пай': 'chocopie',
            'мос.кар.': 'московский картофель',
            'maxib.': 'maxibon',
            'р.хол.': 'русский холод',
            'extr.': 'extreme',
            'свитл.': 'свитлогорье',
            'дом.кух.': 'домашняя кухня',
            'дом кух': 'домашняя кухня',
            'китекат': 'kitekat',
            'контекс': 'contex',
            'лукоил': 'lukoil',
            'орбит': 'orbit',
            'пепси': 'pepsi',
            'простокваш': 'простоквашино',
            'простоквашино': 'простоквашино',
        }
        self.brands = ['магнит', 'foxlite', 'пятерочка', 'домик в деревне', 'перекресток', 'карусель', 'лента', 'я самая',
                      'billa', 'мягкий знак', 'окей', 'саф-момент', 'рот-фронт', 'му-у', 'm&ms', 'брест-литовский', 'oral-b',
                      'mccormick', 'хлебцы-молодцы', 'coca-cola', 'агро-альянс', 'eco-botanica', 'агуша', 'фрутоняня', 'heinz',
                       'мираторг', 'вкусвилл', 'danone', 'bonduelle', 'hochland', 'greenfield', 'nescafe', 'president', 'бабаевский',
                      'доширак', 'lorenz naturals', 'colgate', 'barilla', 'tolli', 'ролтон', 'юбилейное', 'aura family', 'astoria', 'из углича',
                      'моя цена', 'коммунарка', 'хлебозавод', 'орловский', 'страна васильки', 'дикси', 'первым делом', 'мера вкуса', 'щелковский',
                      'кириешки', 'кузя', 'крестьянское', 'яков давыдов', 'волковская пивоварня', 'дмитровский', 'российский',
                      'завтрак+', 'первак', 'океан', 'маракас', 'круглый год', 'фруто-няня', 'зеленая линия', 'овсяная изюминка', 'наше солнышко',
                      'три корочки', 'тульский', 'особый', 'гиагинский', 'краски лета', 'кока кола', 'маруся', 'фрекен бок', 'индана', 'малиновка', 'любодарово', 'белая долина',
                        'шишкин лес', 'юрма', 'марьин', 'топ шеф', 'красна ягода', 'восточный гость', 'молти', 'праздничная', 'томмолоко', 'орион', 'тараллини',
                       'dirol', 'варвара краса', 'море рядом', 'как раньше', 'цукерман', 'донской табак', 'сиббиттер', 'белевская пастила', 'бифилайф',
                       'степановна', 'отличная цена', 'kotex', 'мясной дом', 'коптильный двор', 'каприз', 'ашан', 'ля фам', 'родные просторы',
                       'даниссимо', 'провансаль', 'царский вкус', 'доброфлот', 'pepsi',
                       
                      ]
        
    @staticmethod
    def remove_all_words_which_contains_number(name, product, brand):
        # Todo: Создать словарь с брендами, у которых есть цифры, например, 365 ДНЕЙ, 7 DAYS
        brands_with_numeric = [
            '365 дней', '7 days', '6 соток', '5 морей', '4 сезона', '7up', '3 glocken', '1toy', '48 копеек',
            'j7', 'got2b', 'хлебозавод 28', '4life','4 life' '36 копеек', 'байкал', '101 зерно', '5 океанов', 'j-7',
            '48коп', 'петр I'
        ]
        
        for br in brands_with_numeric:
            if br in name:
                brand = br
                if brand == '48коп':
                    brand = '48 копеек'
                name = name.replace(br, '')
                break
        name = ' '.join(re.sub(r'\w*\d\w*', '', word).strip() for word in name.split())
        return pd.Series([name, product, brand])
    
    def remove_all_words_which_contains_slash(self, name, product, brand):
        for key, value in self.slash_brands.items():
            if key in name:
                brand = value
                name = name.replace(key, ' ')
                break
        
        for key, value in self.slash_product.items():
            if key in name:
                product = value
                name = name.replace(key, ' ')
                break
        name = name.replace('/', ' ')
#         name = ' '.join(re.sub(r'\w*/\w*', ' ', word).strip() for word in name.split())
        return pd.Series([name, product, brand])
    
    def remove_punctuation_symbols(self, name, product, brand):
        for key, value in self.dot_brands.items():
            if key in name:
                brand = value
                name = name.replace(key, '')
                break
        
        tmp = [re.sub(r'([.,!?%:№*\(|\)])', ' ', word).strip() for word in name.split()]
        name = ' '.join(word[:-1] if word.endswith('-') else word for word in tmp)
        return pd.Series([name, product, brand])
    
    @staticmethod
    def remove_one_and_two_chars(name: str):
#         whitelist = ['с', 'в', 'и', 'из']
        whitelist = ['fa']
        return ' '.join(x.strip() for x in name.split() if len(x) > 2 or x in whitelist)
#         return ' '.join(re.sub(r'\b\w{,2}\b', ' ', word).strip() for word in name.split())
    
    
    def find_brands(self, name, product, brand):
        brands_v2 = {
            'greenf': 'greenfield',
            'hochl': 'hochland',
            'nesc': 'nescafe',
            'ч лин': 'чистая линия',
            'pres': 'president',
            'бабаев': 'бабаевский',
            'дош': 'доширак',
            'бр-лит': 'брест-литовский',
            'lor': 'lorenz naturals',##### ?
            'dan': 'danone',
            'bond': 'bonduelle',
            'colg': 'colgate'
        }
        for br in self.brands:
            if br in name:
                brand = br
                name = name.replace(br, '')
                break
    
        for key, value in brands_v2.items():
            if key in name:
                brand = value
                name = name.replace(key, '')
                break
        return pd.Series([name, product, brand])
    
    @staticmethod
    def remove_words_in_blacklist(name: str):
        blacklist = ['бзмж', 'вес', 'для', 'нарезной', 'пшен', 'тенд', 'белый', 'premium', 'декор', 'ранний',
                    'струч', 'короткоплодные', 'венские', 'хлебопек', 'репчатый', 'упак', 'мытая',
                    'дуболистный', 'красный', 'газовая', 'средний', 'слив', 'перс', 'груш', 'классич', 'прод',
                     'раств', 'зерно', 'ябл-бан', 'выгодно', 'красные', 'цветы', 'раст', 'кета', 'косметик',
                    'короткоплодн', 'абр', 'жел', 'шейкер', 'гречн', 'сдоб', 'н-р', '-сл', '[м]', 'вбд',
                     'россия', 'по-домашнему', 'по-корейски', 'киш-миш', 'тд-холдинг', 'дой-пак', 'фрито',
                    'кисло-сладкий', 'рж-пш', 'оквашино', 'нар', 'фас', 'изд', 'охл', 'бел', 'дет', 'паст', 'one',
                    'сливочный', 'зеленью', 'плавл', 'ассорти', 'классическое', 'суп&соус', 'чизбургер', 'ломтик', 'чизбург',
                    'пец-хаас', 'кукурузный', 'ванильный', 'пищевой', 'баварская', 'мульт', 'крупные', 'оливлайн',
                     'черные', 'val', 'плав', 'ломт', 'асс', 'фин', 'избр', 'плавленый', 'финский', 'творожный',
                    'рисунком', 'арт', 'р-р', 'гладкие', 'крас', 'премиум', 'лбп', 'пачка', 'сушеные', 'хлебзернопрод', 'азербайджан', 'розовые',
                    'смо', 'изо', 'ивиа', 'прозр', 'барное', 'светлое', 'темное', 'ультр', 'среднеплодные', 'гладк', 'прополис', 'чесноком', 'карам',
                    'детский', 'яблоко-вишня', 'ишка', 'фасованные', 'детское', 'ржаной', 'украинск', 'оливковый', 'инд', 'окваш', 'моц', 'столовое', 'бокс',
                    'молочный', 'нарез', 'столовая', 'весенние', 'зимой', 'зер', 'зерненый', 'бород', 'белокочанная', 'ранняя', 'домашние', 'морская', 'мелкая', 'рус', 'кривское',
                    'классический', 'ригли', 'вал', 'дрож', 'ябл-черная', 'соев', 'ван', 'москов', 'докторская', 'сух', 'дрожжевое', 
                     'слоеное', 'кокосом', 'зеленый', 'спм', 'вареная', 'выпечки', 'курин', 'пшеничная', 'мытый', 'берканские', 'сладкие',
                    'навелин', 'круглый', 'пинк леди', 'желтый', 'вильямс', 'топл', 'белые', 'спз', 'кдв', 'цыпл-брой', 'сетке',
                     'длинноплодный', 'соленые', 'ский', 'бок', 'тер', 'овские', 'атл', 'маринован', 'гов', 'бородинский', 'рожки', 'пинк', 'хлебоп', 'адриатика',
                    'черн-овес', 'пмм', 'жёлтый', 'дск', 'ветке', 'колотый', 'мга', 'домашний', 'подложка', 'цена', 'детские', 'палермо', 'ромашка', 'фетакса', 'отб',
                    'горький', 'сметанковыймдж', 'комсомольская правда', 'сальчичон', 'гос', 'горшочке', 'красна ягода', 'самокл', 'каймак', 'мягкий', 'кос', 'завар',
                    'шок-ор', 'литовский', 'семейка', 'парижский', 'шашл', 'отборный', 'бакинские', 'отб', 'свежий', 'нарезка', 'форелл', 'стер', 'экон', 'нарезка',
                    'голден', 'дров', 'гладкий', 'больш', 'бриут', 'даниловский', 'франц', 'рислинг', 'тсх', 'тепличные', 'овские', 'стол ']
        return ' '.join(word for word in name.split() if word.strip() not in blacklist)
    
    @staticmethod
    def replace_with_product_dict(name: str):
        return ' '.join(PRODUCT_DICT.get(word.strip(), word) for word in name.split())
    
    @staticmethod
    def replace_most_commen_reductions(name: str):
        reductions = {
            'ср-во': 'средство',
            'филе-кусок': 'филе',
            'нап': 'напиток',
            'крем-мыло': 'мыло',
            'мини-сушки': 'сушки',
            'лук-репка': 'лук',
            'отбел': 'отбеливатель',
            'биотвор': 'биотворог',
            'биойог': 'биойогурт',
            'вин': 'вино',
            'припр': 'приправа',
            'мол': 'молоко',
            'шок': 'шоколад',
            'конф': 'конфеты',
            'жев': 'жевательная резинка',
            'колб': 'колбаса',
            'пакет-майка': 'пакет',
            'твор': 'творог',
            'печен': 'печенье',
            'штруд': 'штрудель',
            'пашт': 'паштет',
            'твор': 'творог',
            'йог': 'йогурт',
            'морож': 'мороженое',
            'салф': 'салфетки',
            'рез': 'резинка',
            'бат': 'батон',
            'майон': 'майонез',
            'марм': 'мармелад',
            'дес': 'десерт',
            'жеврезинка': 'жевательная резинка',
            'шамп': 'шампанское',
            'шокол': 'шоколад',
            'лим кислота': 'лимонная кислота',
            'том.паста': 'томатная паста',
            'ваф': 'вафли',
            'пельм': 'пельмени',
            'спагет': 'спагетти',
            'мюс': 'мюсли',
            'кор': 'коре',
            'наполн': 'наполнитель',
            'пирож': 'пирожное',
            'макар': 'макароны',
            'порош': 'порошок',
            'tоматы': 'томаты',
            'сред': 'средство',
            'свечи-мини': 'свечи',
            'проклад': 'прокладки',
            'хлоп': 'хлопья',
            'шам': 'шампунь',
            'нект': 'нектар',
            'зп': 'зубная паста',
            'сухар': 'сухари',
            'сосис': 'сосиски',
            'приправ': 'приправа',
            'котл': 'котлета',
            'прокл': 'прокладки',
            'салфет': 'салфетки',
            'мармел': 'мармелад',
            'печ': 'печенье',
            'напит': 'напиток',
            'багета': 'багет',
            'горош': 'горошек',
            'мак': 'макароны',
            'семеч': 'семечки',
            'кренд': 'крендели',
            'пасх': 'пасха',
            'зажиг': 'зажигалка',
            'батонч': 'батончик',
            'сырчечилспаг': 'сыр',
            'ароматиз': 'ароматизатор',
            'наб': 'набор',
            'антиперсп': 'антиперспант',
            'бальз-опол': 'бальзам',
            'бзмжсыр': 'сыр',
            'био-йогу': 'биойогурт',
            'био-йогурт': 'биойогурт',
            'биотв': 'биотворог',
            'бумаг': 'бумага',
            'вермиш': 'вермишель',
            'гель-эксп': 'гель',
            'зажигалок': 'зажигалка',
            'кетч': 'кетчуп',
            'кнофеты': 'конфеты',
            'крем-мыл': 'мыло',
            'кэфир': 'кефир',
            'лосьон-тон': 'лосьон',
            'макарон': 'макароны',
            'манг': 'манго',
            'морожен': 'мороженое',
            'пирожн': 'пирожное',
            'подг': 'подгузники',
            'презерв': 'презервативы',
            'сосиск': 'сосиски',
            'тамп': 'тампоны',
            'наб': 'набор',
            'биойог': 'биойогурт',
            'кетч': 'кетчуп',
            'кукур': 'кукуруза',
            'бум': 'бумага',
            'разрыхлит': 'разрыхлитель',
            'красит': 'краситель',
        }
        return ' '.join(reductions.get(word.strip(), word) for word in name.split())
        
    def find_scratch_card(self, name, product, brand):
        if 'скретч-карта' in name:
            return pd.Series(['', 'скретч-карта', 'перекресток'])
        return pd.Series([name, product, brand])
    
    @staticmethod
    def find_product(name, product):
        if not product:
            if len(name.split()) == 1:
                if name == 'мол':
                    return pd.Series(['', 'молоко'])
                product = name
                return pd.Series(['', product])
            return pd.Series([name, None]) 
        return pd.Series([name, product])

    @staticmethod
    def find_and_remove_all_english_words(name, brand):
        eng_brands = re.findall(r'\b([a-z]+)\b', name)
        if eng_brands and not brand:
            result = []
            for word in eng_brands:
                name = name.replace(word, '').replace('  ', ' ').strip()
                result.append(word)

            brand = ' '.join(result)
            if 'epica' in brand:
                brand = 'epica'
            return pd.Series([name, brand])

        # If we have brand, we should delete english words in product name
        name = re.sub(r'\b([a-z]+)\b', '', name).replace('  ', ' ').strip()
        return pd.Series([name, brand])
    
    def normalize(self):
        self.df['name_norm'] = self.df['name_norm'].str.lower()
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_all_words_which_contains_number(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_all_words_which_contains_slash(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df['name_norm'] = self.df['name_norm'].apply(self.replace_with_product_dict)
        
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_punctuation_symbols(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_brands(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df['name_norm'] = self.df['name_norm'].apply(self.remove_one_and_two_chars)
        self.df['name_norm'] = self.df['name_norm'].apply(self.remove_words_in_blacklist)
        self.df['name_norm'] = self.df['name_norm'].apply(self.replace_most_commen_reductions)
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_scratch_card(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_and_remove_all_english_words(x['name_norm'], x['brand_norm']), axis=1
        )
#         self.df[['name_norm', 'product_norm']] = self.df.apply(
#             lambda x: self.find_product(x['name_norm'], x['product_norm']), axis=1
#         )
        return self.df

In [148]:
%%time
n = NormalizeNames(real['name'])
tmp = n.normalize()
# Wall time: 1min 24s

CPU times: user 1min 5s, sys: 46.9 ms, total: 1min 5s
Wall time: 1min 5s


In [149]:
df = pd.concat([real, tmp], axis=1)[['shop_name', 'name', 'name_norm', 'product_norm', 'brand_norm']]

In [150]:
df.brand_norm.dropna().shape # 13012

(13012,)

In [151]:
df.product_norm.dropna().shape # 365

(365,)

In [153]:
from typing import Optional
from itertools import combinations
from typing import List
# import pymorphy2
from pymystem3 import Mystem


class FinderInEtalon:
    def __init__(self, df: pd.DataFrame, path_to_brands: str = 'clean_data/rus_brands.csv',
                 path_to_products: str = 'clean_data/products.csv', path_to_etalon='clean_data/all_clean.csv'):
        self.rus_brands = pd.read_csv(path_to_brands)['Бренд'].values
        self.products = pd.read_csv(path_to_products)
        self.clean = pd.read_csv(path_to_etalon)
        self.columns = ['name_norm', 'product_norm', 'brand_norm']
        self.df = df[self.columns].copy()
        self.df['cat_norm'] = None
        self.mystem = Mystem()
        
    def find_brands(self, name: str, brand: Optional[str] = None) -> pd.Series:
        if name and not brand:
            names = set([f'{comb[0]} {comb[1]}' for comb in combinations(name.split(), 2)] + name.split())
            for rus_brand in self.rus_brands:
                if rus_brand in names:
                    name = name.replace(rus_brand, '').replace('  ', ' ').strip()
                    return pd.Series([name, rus_brand])
        return pd.Series([name, brand])

    @staticmethod
    def _remove_duplicate_word(arr: List[str]) -> List[str]:
        """
        Remove duplicate words from products:
        ['вода', 'вода питьевая'] --> ['вода питьевая']
        """

        if max([len(x.split()) for x in arr]) > 1:
            arr = sorted(arr, key=lambda x: len(x.split()))
            one_words = []
            for product in arr.copy():
                if len(product.split()) == 1:
                    one_words.append(product)
                else:
                    for word in one_words:
                        if word in product:
                            arr.remove(word)
        return arr

    def find_product(self, name: str, product: str, cat: Optional[str]=None):
        if name and not product:
            names = pd.DataFrame(set([f'{comb[0]} {comb[1]}' for comb in combinations(name.split(), 2)] + name.split()), columns=['Продукт'])
            merge = self.products.merge(names)
            if len(merge):
                product = ', '.join(self._remove_duplicate_word(merge['Продукт'].values))
                cat = merge['Категория'].value_counts().index[0]
        return pd.Series([name, product, cat])
    
    @staticmethod
    def remove_dash(name: str, product: str) -> str:
        if name and not product:
            name = name.replace('-', ' ')
        return name

    @staticmethod
    def use_pymorphy(name: str, product: str) -> str:
        if name and not product:
            name = ' '.join(morph.parse(x)[0].normal_form for x in name.split())
        return name
    
    def use_mystem(self, name: str, product: str) -> str:
        if name and not product:
            name = ''.join(self.mystem.lemmatize(name)[:-1])
        return name
    
    def find_category(self, product: str, cat: str) -> pd.Series:
        if product and not cat:
            tmp = self.products[self.products['Продукт'] == product]
            if len(tmp):
                cat = tmp['Категория'].values[0]

        return pd.Series([product, cat])


    def find_product_by_brand(self, product: str, brand: str, cat: str) -> pd.Series:
        if not product and brand:
            tmp = self.clean[self.clean['Бренд'] == brand]
            if len(tmp):
                product = tmp['Продукт'].value_counts().index[0]
                cat = tmp['Категория'].value_counts().index[0]

        return pd.Series([product, brand, cat])
    
    
    def print_logs(self)-> None:
        print(
            "Recognized brands: "
            f"{len(self.df['brand_norm'].dropna())}, "
            f"products: {len(self.df['product_norm'].dropna())}, "
            f"categories: {len(self.df['cat_norm'].dropna())}",
            '-'*60, sep='\n', end='\n\n'
        )
    
    def find_all(self):
        
        print('Before:')
        self.print_logs()
        
        # Find brand:
        print('Find brands...')
        self.df[['name_norm', 'brand_norm']] = self.df.apply(lambda x: self.find_brands(x['name_norm'], x['brand_norm']), axis=1)
        self.print_logs()
        
        # Find product and category:
        print('Find product...')
        self.df[['name_norm', 'product_norm', 'cat_norm']] = self.df.apply(lambda x: self.find_product(x['name_norm'], x['product_norm']), axis=1)
        self.print_logs()
        
        # Remove `-` and the second attempt to find a product:
        print('Remove `-`...')
        self.df['name_norm'] = self.df.apply(lambda x: self.remove_dash(x['name_norm'], x['product_norm']), axis=1)
        self.df[['name_norm', 'product_norm', 'cat_norm']] = self.df.apply(lambda x: self.find_product(x['name_norm'], x['product_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        
        # Use Mystem for lemmatization and the third attempt to find a product:
        print('Lemmatization...')
        self.df['name_norm'] = self.df.apply(lambda x: self.use_mystem(x['name_norm'], x['product_norm']), axis=1)
        self.df[['name_norm', 'product_norm', 'cat_norm']] = self.df.apply(lambda x: self.find_product(x['name_norm'], x['product_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        
        # Find category:
        print('Find category...')
        self.df[['product_norm', 'cat_norm']] = self.df.apply(lambda x: self.find_category(x['product_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        
        # Find product by brand:
        print('Find product by brand...')
        self.df[['product_norm', 'brand_norm', 'cat_norm']] = self.df.apply(lambda x: self.find_product_by_brand(x['product_norm'], x['brand_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        return self.df

In [154]:
%%time
finder = FinderInEtalon(df)
res = finder.find_all()
res = pd.concat([df['name'], res], axis=1)
# Recognized brands: 17178, products: 30343, categories: 30343

# Recognized brands: 17358, products: 30499, categories: 30499 split by '/'

# Wall time: 3min 19s

Before:
Recognized brands: 13012, products: 365, categories: 0
------------------------------------------------------------

Find brands...
Recognized brands: 17358, products: 365, categories: 0
------------------------------------------------------------

Find product...
Recognized brands: 17358, products: 29465, categories: 29100
------------------------------------------------------------

Remove `-`...
Recognized brands: 17358, products: 29563, categories: 29198
------------------------------------------------------------

Lemmatization...
Recognized brands: 17358, products: 29929, categories: 29564
------------------------------------------------------------

Find category...
Recognized brands: 17358, products: 29929, categories: 29929
------------------------------------------------------------

Find product by brand...
Recognized brands: 17358, products: 30499, categories: 30499
------------------------------------------------------------

CPU times: user 3min 17s, sys: 469 ms, 

In [122]:
def find_category(product: str, cat: str) -> str:
    if product and not cat:
        tmp = products[products['Продукт'] == product]
        if len(tmp):
            cat = tmp['Категория'].values[0]
            return pd.Series([product, cat])
    return pd.Series([product, cat])


def find_product_by_brand(product: str, brand: str, cat: str) -> pd.Series:
    if not product and brand:
        tmp = clean[clean['Бренд'] == brand]
        if len(tmp):
            product = tmp['Продукт'].value_counts().index[0]
            cat = tmp['Категория'].value_counts().index[0]
        
    return pd.Series([product, brand, cat])

In [129]:
q = res.apply(lambda x: find_product_by_brand(x['product_norm'], x['brand_norm'], x['cat_norm']), axis=1)

In [130]:
q

,0,1,2
19,"филе, минтай",None,"Птица, мясо, деликатесы"
20,молоко,страна васильки,"Молоко, сыр, яйца"
21,сушки,тараллини,"Хлеб, сладости, снеки"
22,яйцо куриное,None,"Хлеб, сладости, снеки"
23,печенье,юбилейное,"Хлеб, сладости, снеки"
...,...,...,...
74012,сыр,российский,"Молоко, сыр, яйца"
74013,колбаса,None,"Птица, мясо, деликатесы"
74015,сырок,б.ю. александров,"Молоко, сыр, яйца"
74017,карбонад,велком,"Птица, мясо, деликатесы"


In [132]:
q[(q[1].apply(lambda x: bool(x))) & (q[0].apply(lambda x: not bool(x)))]

,0,1,2
555,None,экстра,None
628,None,trio,None
742,None,geese,None
2124,None,perfect,None
3418,None,guandy,None
...,...,...,...
72087,None,winston,None
72512,None,firemark,None
72591,None,bake easy,None
72962,None,wick pure,None


In [136]:
q.loc[73984]

0                   хлебцы
1           хлебцы-молодцы
2    Хлеб, сладости, снеки
Name: 73984, dtype: object

In [126]:
res[(res['brand_norm'].apply(lambda x: bool(x))) & (res['product_norm'].apply(lambda x: not bool(x)))]

,name,name_norm,product_norm,brand_norm,cat_norm
284,БЗМЖ МАЦОНИ КОРОВКА ИЗ КОРЕНОВ,мацони корен,None,коровка,None
363,СУХ/КОРМ KITEKAT АППЕТИТНАЯ ТЕ,аппетитный,None,kitekat,None
366,ИВАСИ КОПЧЕНАЯ ДОБРОФЛОТ Ж/Б 2,иваси копченый,None,доброфлот,None
555,ГРЕЧА ЭКСТРА ЭЛИТНАЯ 900Г,греча элитный,None,экстра,None
591,КОНД-Р Д/БЕЛЬЯ LENOR ДЕТСКИЙ К,конд р,None,lenor,None
...,...,...,...,...,...
72761,Парламент Найт Блю пачка,найт блю,None,парламент,None
72962,Баллон сменныйAir Wick Pure к,баллон сменныйair,None,wick pure,None
73052,Винстон Компакт Импульс Саммер,компакт импульс саммер,None,winston,None
73333,Собиратель пыли антистатик Rozenbal,собиратель пыль антистатик,None,rozenbal,None


In [109]:
clean

,Название,Категория,Бренд,Вес,Продукт
0,"le petit marseillais гель-шампунь, апельсиново...","Красота, гигиена, бытовая химия",le petit marseillais,NaN,гель
1,"антиперспирант спрей axe, ледокол, 150 мл","Красота, гигиена, бытовая химия",axe,150,антиперспирант
2,"антиперспирант- спрей dove, нежность лепестков...","Красота, гигиена, бытовая химия",dove,NaN,антиперспирант
3,"бальзам для волос schauma, детоксматча, 200 мл","Красота, гигиена, бытовая химия",schauma,200,бальзам для волос
4,бальзам-ополаскиватель пантин интенсивное восс...,"Красота, гигиена, бытовая химия",pantene,NaN,бальзам
...,...,...,...,...,...
71633,набор игровой paula спортсмен сёрфинг,Товары для мам и детей,paula,0.12 кг,набор игровой
71634,набор игровой paula транспорт,Товары для мам и детей,paula,0.48 кг,набор игровой
71635,набор игровой paula моя большая ферма,Товары для мам и детей,paula,0.85 кг,набор игровой
71636,подгузники merries для детей размер м 6-11кг 76шт,Товары для мам и детей,merries,2.3 кг,подгузники


In [121]:
clean[clean['Бренд'] == 'axe']

,Название,Категория,Бренд,Вес,Продукт
1,"антиперспирант спрей axe, ледокол, 150 мл","Красота, гигиена, бытовая химия",axe,150,антиперспирант
701,дезодорант axe аэрозольный для мужчин gold,"Красота, гигиена, бытовая химия",axe,150 мл,дезодорант
2401,антиперспирант axe urban шариковый защита от з...,"Красота, гигиена, бытовая химия",axe,"0,15 кг",антиперспирант
2402,антиперспирант axe signature gold шариковый за...,"Красота, гигиена, бытовая химия",axe,"0,14 кг",антиперспирант
2403,антиперспирант axe adrenaline шариковый усилен...,"Красота, гигиена, бытовая химия",axe,"0,15 кг",антиперспирант
...,...,...,...,...,...
70357,"шампунь axe для волос anarchy 0,25л","Красота, гигиена, бытовая химия",axe,0.298 кг,шампунь
70359,шампунь-кондиционер axe для волос 2в1 apollo 0...,"Красота, гигиена, бытовая химия",axe,0.298 кг,шампунь-кондиционер
70364,антиперспирант axe ledokol шариковый 50мл,"Красота, гигиена, бытовая химия",axe,0.09 кг,антиперспирант
70396,антиперспирант axe ledokol карандаш 50мл,"Красота, гигиена, бытовая химия",axe,0.109 кг,антиперспирант


In [98]:
products[products['Продукт'] == 'зубная паста']['Категория'].values[0]

'Красота, гигиена, бытовая химия'

In [101]:
res[(res['product_norm'].apply(lambda x: bool(x))) & (res['cat_norm'].apply(lambda x: not bool(x)))].apply(lambda x: find_category(x['product_norm'], x['cat_norm']), axis=1)

384      Красота, гигиена, бытовая химия
451      Красота, гигиена, бытовая химия
596      Красота, гигиена, бытовая химия
607      Красота, гигиена, бытовая химия
1041     Красота, гигиена, бытовая химия
                      ...               
71054                  Молоко, сыр, яйца
71056                  Молоко, сыр, яйца
71057                  Молоко, сыр, яйца
71762               Овощи, фрукты, ягоды
72365               Овощи, фрукты, ягоды
Length: 365, dtype: object

In [71]:
df.head()

,name,name_norm,product_norm,brand_norm,cat_norm
19,МИНТАЙ ФИЛЕ Б/К СВ/МОР П/ПАК 8,минтай филе,"филе, минтай",None,None
20,БЗМЖ МОЛОКО СТРАНА ВАСИЛЬКИ У/,молоко,молоко,страна васильки,None
21,СУШКИ ТАРАЛЛИНИ С ЧЕСНОКОМ 180,сушки,сушки,тараллини,None
22,ЯЙЦО КУРИНОЕ СТОЛОВОЕ 1КАТЕГОР,яйцо куриное,яйцо куриное,None,None
23,ПЕЧЕНЬЕ ЮБИЛЕЙНОЕ ВИТАМИНИЗИРО,печенье витаминизиро,печенье,юбилейное,None


In [64]:
df.sample(10)

,name,name_norm,product_norm,brand_norm,cat_norm
30944,*79051 WAG.W.Печен.с суф/дж.гл.228г,печенье,печенье,wag,None
10748,6: 3477315 Соус FILIPPO BERIO ПЕСТО 190г,соус песто,соус,filippo berio,None
9131,СНЕЖНОЕ ЛАКОМСТВО Морож брикет пломбир ваниль 80,снежное лакомство мороженое брикет пломбир ваниль,"лакомство, мороженое пломбир",None,None
42761,"КАГОР ЮЖНОБЕРЕЖ 0,75",кагор южнобережа,None,None,None
22132,5:3438858 Хлебцы СТОЙЛЕНСКАЯ НИВА 100г,хлебцы стойленская нива,хлебцы,None,None
17416,1*3926617 Подушечки NESQUIK 220г,подушечки,подушечки,nesquik,None
41026,"ТУШКА ЦБ 1С.ОХЛ,КД",тушка,тушка,None,None
2829,БЗМЖ СЫР ПЛ.PRESIDENT/ПРЕЗИД.С,сыр,сыр,None,None
60993,Сырок твор ТВОРОБУШКИ глаз конд гл21%180,сырок творог глазурь,"творог, сырок, глазурь",творобушки,None
41134,СОУС С КР.ПЕРЦЕМ 60М,соус перцем,соус,None,None


### TODO:

5. Для тех записей, где нет продукта, но есть бренд - подставлять частовстречаемый продукт по бренду.
6. Найти категории для уже найденных продуктов


In [88]:
from itertools import combinations
from typing import List

products = pd.read_csv('clean_data/products.csv')

def remove_duplicate_word(arr: List[str]) -> List[str]:
    """
    Remove duplicate words from products:
    ['вода', 'вода питьевая'] --> ['вода питьевая']
    """
    
    if max([len(x.split()) for x in arr]) > 1:
        arr = sorted(arr, key=lambda x: len(x.split()))
        one_words = []
        for product in arr.copy():
            if len(product.split()) == 1:
                one_words.append(product)
            else:
                for word in one_words:
                    if word in product:
                        arr.remove(word)
    return arr

def find_product(name: str, product: str):
    if name and not product:
        names = pd.DataFrame(set([f'{comb[0]} {comb[1]}' for comb in combinations(name.split(), 2)] + name.split()), columns=['Продукт'])
        merge = products.merge(names)
        if len(merge):
            product = ', '.join(remove_duplicate_word(merge['Продукт'].values))
            category = merge['Категория'].value_counts().index[0]
            return pd.Series([name, product, category])
    return pd.Series([name, product, None])

def remove_dash(name: str, product: str) -> str:
    if name and not product:
        name = name.replace('-', ' ')
    return name

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def use_pymorphy(name: str, product: str) -> str:
    if name and not product:
        name = ' '.join(morph.parse(x)[0].normal_form for x in name.split())
    return name

In [26]:
%%time
res[['name_norm', 'product_norm', 'cat_norm']] = res.apply(lambda x: find_product(x['name_norm'], x['product_norm']), axis=1)
# Wall time: 1min 37s


CPU times: user 1min 34s, sys: 109 ms, total: 1min 34s
Wall time: 1min 35s


In [27]:
res = pd.concat([df['name'], res], axis=1)

In [28]:
res.product_norm.dropna().shape # 9343 -> 27570 -> 28930 -> 29617 -> 28684 -> 28744 -> 29316 -> 29455 -> 29496 -> 29821

(29821,)

In [69]:
res['name_norm'] = res.apply(lambda x: remove_dash(x['name_norm'], x['product_norm']), axis=1)

In [71]:
res[['name_norm', 'product_norm', 'cat_norm']] = res.apply(lambda x: find_product(x['name_norm'], x['product_norm']), axis=1)

In [56]:
res.product_norm.dropna().shape # 9343 -> 27570 -> 28930 -> 29617 -> 28684 -> 28744 -> 29316 -> 29455 -> 29496 -> 29821 -> 29907

(29743,)

In [85]:
res['name_morphy'] = res.apply(lambda x: use_pymorphy(x['name_norm'], x['product_norm']), axis=1)

In [91]:
res[res['name_morphy'].apply(lambda x: bool(x))].sample(10)

,name,name_norm,product_norm,brand_norm,cat_norm,name_morphy
1079,БЗМЖ МОРОЖЕН.КАК РАНЬШЕ ПЛОМБИ,мороженое пломби,мороженое,как раньше,None,мороженое пломби
12519,"5*2100611 Батончик BOUNTY ТРИО 82,5г",батончик трио,батончик,bounty,None,батончик трио
1535,ФРУКТОВЫЕ КУСОЧКИ ФРУТОНЯНЯ ЯБ,фруктовые кусочки,None,фрутоняня,None,фруктовый кусочек
65399,Мидии МИЛЕГРИН отб в масле с пров тр415г,мидии милегрин масле пров,мидии,None,None,мидии милегрин масле пров
42466,"КЕФИР ДЕТСК 3,2%200Г",кефир детск,кефир,None,None,кефир детск
73771,Голень индейки охлажденная ТЧН,голень индейки охлажденная тчн,голень,None,None,голень индейки охлажденная тчн
31632,63218 ИНДАНА Приправа для мяса 15г,приправа мяса,приправа,индана,None,приправа мяса
17347,24:14811 Томаты сливовидные 1кг,томаты сливовидные,томаты,None,None,томаты сливовидные
17049,5*: 15105 GALBANI Сыр МОЦАРЕЛЛА 45% 125г,сыр моцарелла,"сыр, моцарелла",galbani,None,сыр моцарелла
42100,КИНЗА ЛОТОК 50 ГРАММ,кинза лоток грамм,"лоток, кинза",None,None,кинза лоток грамм


In [100]:
res[['name_norm', 'product_norm', 'cat_norm']] = res.apply(lambda x: find_product(x['name_morphy'], x['product_norm']), axis=1)

In [101]:
res.product_norm.dropna().shape # 9343 -> 27570 -> 28930 -> 29617 -> 28684 -> 28744 -> 29316 -> 29455 -> 29496 -> 29821 -> 29907 -> 30182

(30182,)

In [40]:
df.sample(10)

,shop_name,name,name_norm,product_norm,brand_norm
23016,ПЕРЕКРЕСТОК,2:3663839 Апельсин отборный 1кг,апельсин,None,None
61869,ЛЕНТА,Ж/мыло ЗЕЛ АПТЕКА Роза и хлопок460,зел аптека роза,жидкое мыло,None
46345,АШАН,ЙОГ ГРЕЧЕСКИЙ НАТ 2%140Г,йогурт греческий,None,None
7425,МАГНИТ,"ФРУТОНЯНЯ Кисель малина 0,2л т/пак(П",кисель малина,None,фрутоняня
65634,ЛЕНТА,Горчица BORNIER Dijon Senf Дижонск 370г,горчица дижонск,None,bornier dijon senf
9233,МАГНИТ,СЕМЕЧЕК СТАКАН Семена Подсолнечн Обж 100г фл/п (,семечек стакан семена подсолнечн обж,None,None
44404,АШАН,САХАРНАЯ ВАТА В ВЕД,сахарная вата вед,None,None
15378,ПЕРЕКРЕСТОК,1*3645971 Смесь GRAND MASTER 550г,смесь,None,grand master
50812,БИЛЛА,СЕМЕНА,семена,None,None
16535,ПЕРЕКРЕСТОК,3694936 ЭКОМ.Масло слив.шок.62% 100г,масло шоколад,None,ecomilk


In [153]:
res[(res['product_norm'].apply(lambda x: not bool(x))) & (res['brand_norm'].apply(lambda x: bool(x)))].sample(10)

,name,name_norm,product_norm,brand_norm,cat_norm,name_morphy
26436,*3490517 R.SP.Шок.ЦЕЛ.ЛЕС.ОРЕХ/ХЛ.б.100г,шоесть лес,None,красная цена,None,шоесть лес
65415,Клейкая лента 365 ДНЕЙ двустор 48ммх10м,клейкий двустор,None,лента,None,клейкий двустор
50256,"PEPSI MAX 1,5Л",,None,pepsi max,None,
47414,КД/ПЛАТОЧКИ,,None,каждый день,None,
15410,3498894 HALLS Карам.ОРИГ.лед.мент/эв25г,орига лёд,None,halls,None,орига лёд
34761,3900912 ШАРЛ Мини круасс.нач.Клуб.200г,минь круасс нач клуб,None,шарлиз,None,минь круасс нач клуб
16803,4*: 3641934 HALLS Карам.лед.со вк.клуб.2,лёд клуб,None,halls,None,лёд клуб
5443,КРАСКИ ЛЕТА Соте 400 г п/уп(Русская овощная комп,соте русский овощной комп,None,краски лета,None,соте русский овощной комп
46426,"ТЕТР.УЧ,24Л ЛИНЕЙКА",тетр уейка,None,чистая линия,None,тетр уейка
29877,3023899 PERS.СМС КОЛОР СВ.ВЕРНЕЛЬ 3кг,смс колор вернель,None,pers,None,смс колор вернель


In [158]:
clean[clean['Бренд'] == 'pepsi']['Продукт'].value_counts()

напиток                 27
газированный напиток    23
напиток газированный     9
Name: Продукт, dtype: int64

In [124]:
res[res['name_norm'] == 'кукура слад зерна']

,name,name_norm,product_norm,brand_norm,cat_norm,name_morphy
19922,4: 3792 BOND.Кукур.слад.зерн.ж/б 340г,кукура слад зерна,None,bonduelle,None,кукура слад зерна
27972,*16793 BOND.Кукур.слад.зерн.ж/б. 212мл,кукура слад зерна,None,bonduelle,None,кукура слад зерна
28054,16793 BOND.Кукур.слад.зерн.ж/б. 212мл,кукура слад зерна,None,bonduelle,None,кукура слад зерна
28285,*3792 BOND.Кукур.слад.зерн.ж/б 340г,кукура слад зерна,None,bonduelle,None,кукура слад зерна
30638,3792 BOND.Кукур.слад.зерн.ж/б 340г,кукура слад зерна,None,bonduelle,None,кукура слад зерна


In [110]:
l = []

def find_category(product: str, cat: str) -> str:
    if product and not cat:
        tmp = products[products['Продукт'] == product]
        if not len(tmp):
            l.append(product)
        else:
            return tmp['Категория'].values[0]
    return None

In [45]:
kek = res.apply(lambda x: find_category(x['product_norm'], x['cat_norm']), axis=1)

In [61]:
res[res['product_norm'] == '-кабель']

,name,name_norm,product_norm,brand_norm,cat_norm
20177,1: 4029336 USB-кабель TYC-2 100см,,-кабель,usb tyc,None


In [70]:
from IPython.display import clear_output


black_list = []
white_list = {}

for i, product in enumerate(np.unique(l)):
    print(product, f'{i+1} from {len(np.unique(l))}', sep='')
    in_ = input()
    if in_ == '':
        black_list.append(product)
    else:
        white_list[product] = in_
    clear_output()

In [337]:
res[res['product_norm'].apply(lambda x: not bool(x))].sample(10)

,name,name_norm,product_norm,brand_norm,cat_norm
46115,ОЧИСТ ВЕЛЮР И КОВР,очист велюр ковр,None,None,None
29341,3471063 ХЛЕБОК.Батоны НАРЕЗ.в/с нар380г,хлебок батоны,None,None,None
44037,КД/ПРОСТЫНЯ ЕВРО ПЕР,ыня евро пер,None,простоквашино,None
43013,БИСК КЛ/ВАН БАРНИ,бисквит барни,None,None,None
49372,ПЕЧ ШОКОЛАДНОЕ САХ,шоколадное сах,None,None,None
5792,"Биокефир 2,5% 0,5кг (Пискарёвский МЗ):10",биокефир пискарёвский,None,None,None
43878,??? ?????????? 900?,,None,None,None
44385,РОДНЫЕ ПРОСТОРЫ BАФ,родные оры bаф,None,простоквашино,None
51160,РОЗА Ч/Г ЛИМБО (20ШТ,роза лимбо,None,None,None
43539,"ЛИПЕЦ.МИНЕР.ГАЗ.1,5Л",липец минер газ,None,None,None


In [24]:
q = pd.read_csv('clean_data/products.csv')

In [40]:
q[q['Продукт'] == 'дыня']

,Продукт,Категория


In [151]:
q.drop(index=461).to_csv('clean_data/products.csv', index=False)

In [112]:
q.loc[446]['Категория'] = 'Овощи, фрукты, ягоды'

In [137]:
q.loc[950]

Продукт        изд "мозаика-синтез"
Категория    Товары для мам и детей
Name: 950, dtype: object

In [115]:
q.to_csv('clean_data/products.csv', index=False)

In [284]:
'макароны' in q['Продукт'].values

True

In [60]:
l = ['сгущенка', 'открытка', 'поджарка', 'белок', 'плитка', 'окорочка', 'гвоздика', 'горошек', 'редька', 'рассол', 'пасха', 'кислота лимонная']

In [38]:
q['Категория'].unique()

array(['Красота, гигиена, бытовая химия', 'Хлеб, сладости, снеки',
       'Молоко, сыр, яйца', 'Соусы, орехи, консервы',
       'Макароны, крупы, специи', 'Другое', 'Воды, соки, напитки',
       'Рыба, икра', 'Чай, кофе, сахар', 'Замороженные продукты',
       'Птица, мясо, деликатесы', 'Посуда', 'Овощи, фрукты, ягоды',
       'Товары для мам и детей', 'Товары для дома и дачи',
       'Подборки и готовые блюда', 'Зоотовары', 'Бытовая техника',
       'Алкоголь', 'Дача и гриль'], dtype=object)

In [78]:
tmp = pd.DataFrame.from_dict(white_list, orient='index').reset_index().rename(columns={'index': 'Продукт', 0: 'Категория'})

In [125]:
l = ['яблоко', 'долма', 'разрыхлитель', 'бумага', 'кукуруза']

In [21]:
for i in df.product_norm.dropna().drop_duplicates().values:
    if i not in q['Продукт'].values:
        print(i)

тулетная бумага
шариковая ручка
пучок


In [22]:
tmp = pd.DataFrame(
    [
        ['тулетная бумага', 'Красота, гигиена, бытовая химия'],
        ['шариковая ручка', 'Другое'],
        ['пучок', 'Овощи, фрукты, ягоды'],
    ], columns=['Продукт', 'Категория']
)

In [23]:
pd.concat([q, tmp]).to_csv('clean_data/products.csv', index=False)

In [85]:
q.shape

(1265, 2)

In [88]:
q.drop_duplicates(subset='Продукт').to_csv('clean_data/products.csv', index=False)

In [279]:
from collections import Counter

C = Counter()
def lol(name):
    global C
    for i in name.split():
        C[i]+=1



In [280]:
res[res['product_norm'].apply(lambda x: not bool(x))]['name_norm'].apply(lol);